In [64]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import math
from math import radians, cos, sin, asin, sqrt
import sys

In [46]:
# 左下角经纬度坐标
lb_lon = 121.20120490000001
lb_lat = 31.28175691

# 右上角经纬度坐标
rt_lon = 121.2183295
rt_lat = 31.29339344

# 栅格边长（左下角栅格坐标为 (0, 0) ）
grid_len = 20

In [47]:
# 根据两点经纬度计算两点间距离
def haversine(lon1, lat1, lon2, lat2): # 经度1，纬度1，经度2，纬度2 （十进制度数）  
    """ 
    Calculate the great circle distance between two points  
    on the earth (specified in decimal degrees) 
    """  
    # 将十进制度数转化为弧度  
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])  
  
    # haversine公式  
    dlon = lon2 - lon1   
    dlat = lat2 - lat1   
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2  
    c = 2 * asin(sqrt(a))   
    r = 6371 # 地球平均半径，单位为公里  
    return c * r * 1000  

In [48]:
# 横向最大距离
max_distance_x =  haversine(lb_lon, 0, rt_lon, 0)
# 右上角栅格横坐标
int(max_distance_x / 20)

95

In [49]:
# 纵向最大距离
max_distance_y = haversine(0, lb_lat, 0, rt_lat)
# 右上角栅格纵坐标
int(max_distance_y / 20)

64

In [50]:
# 将地图栅格化，将经纬度坐标转化为栅格坐标，左下角的栅格坐标是 (0, 0) ，右上角的栅格坐标是 (95, 64)
def ll_to_grid(lon, lat):
    # 左下角经纬度坐标，栅格边长
    global lb_lon, lb_lat, grid_len
    X = int(haversine(lon, 0, lb_lon, 0) / grid_len) 
    Y = int(haversine(0, lat, 0, lb_lat) / grid_len)
    return (X, Y)

In [82]:
# 合并两张表，用 data_2g 中的 RNCID_1，CellID_1 与 gongcan_2g 的 RNCID，CellID 匹配，将基站的经纬度信息加到 data_2g 中
def merge_data_gongcan():
    data_2g = pd.read_csv('../raw_data/data_2g.csv')
    gongcan_2g = pd.read_csv('../raw_data/2g_gongcan.csv')
    
    for i in range(1, 8):
        # 换掉 gongcan_2g 的列名用以和 data_2g merge
        gongcan_2g.columns = ['RNCID_' + str(i), 'CellID_' + str(i), 'Lat_' + str(i), 'Lon_'+str(i)]    
        data_2g = pd.merge(data_2g, gongcan_2g, how='left', on=['RNCID_' + str(i), 'CellID_' + str(i)])
        
    # 将 RSSI_1 ~ RSSI_7 的空缺值nan用 -sys.maxsize - 1 来代替
    for j in range(1, 8):
        data_2g['RSSI_' + str(j)] = data_2g['RSSI_' + str(j)].fillna(-sys.maxsize - 1)
    
    # 将其余空缺值nan替换为-1
    data_2g = data_2g.fillna(-1)
    return data_2g

In [83]:
# 根据MR数据的GPS加上栅格ID
def add_gridID(data):
    data['GridID'] = data.apply(lambda x: ll_to_grid(x.Longitude, x.Latitude), axis = 1)
    return data

In [84]:
def main():
    # 合并两张表
    data = merge_data_gongcan()
    # 根据MR数据的GPS加上栅格ID
    data = add_gridID(data)
    
    return data

In [85]:
data = main()

In [90]:
data['Lat_7']

0       31.291805
1       31.289647
2       31.288649
3       -1.000000
4       -1.000000
5       -1.000000
6       -1.000000
7       -1.000000
8       -1.000000
9       -1.000000
10      31.291805
11      31.291805
12      31.291805
13      31.289351
14      -1.000000
15      31.289647
16      31.289647
17      -1.000000
18      -1.000000
19      -1.000000
20      -1.000000
21      -1.000000
22      -1.000000
23      -1.000000
24      -1.000000
25      -1.000000
26      -1.000000
27      -1.000000
28      -1.000000
29      -1.000000
          ...    
6066    -1.000000
6067    -1.000000
6068    -1.000000
6069    -1.000000
6070    -1.000000
6071    -1.000000
6072    -1.000000
6073    -1.000000
6074    -1.000000
6075    -1.000000
6076    -1.000000
6077    -1.000000
6078    -1.000000
6079    -1.000000
6080    -1.000000
6081    -1.000000
6082    -1.000000
6083    -1.000000
6084    -1.000000
6085    31.288361
6086    -1.000000
6087    -1.000000
6088    31.276852
6089    31.276852
6090    -1